In [5]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
from scipy.stats import trim_mean
from sklearn.metrics import mean_squared_error

sys.path.append('..')
from higgs_inference import settings
from higgs_inference.various.utils import format_number

In [6]:
result_dir = '../results/'

# TablePrinter class

In [12]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Skip entirely empty lines
            try:
                if not np.any(np.isfinite(line[2:])):
                    continue
            except TypeError:
                print(line)
                continue
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            bracket = False
            try:
                value = fn(filename, folder)
            except (IOError, ValueError):
                #print('File', filename, 'in folder', folder, 'not found')
                value = np.nan
                
            if isinstance(value, (list, tuple)):
                value, bracket = value
                
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [32]:
def expected_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/mse_logr_' + filename + '.npy')[settings.thetas_train]
    return np.mean((mse_log_r))
                       
def expected_trimmed_mse_log_r(filename, folder='parameterized'):
    mse_log_r = np.load(result_dir + folder + '/trimmed_mse_logr_' + filename + '.npy')[settings.thetas_train]
    return np.mean((mse_log_r))

In [33]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.sqrt(np.median(errors))

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return np.sqrt(mean_squared_error(log_r_truth, log_r_estimated))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(mean_squared_error(log_r_truth[finites], log_r_estimated[finites])), True
    
def trimmed_mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction))
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return np.sqrt(trim_mean((log_r_truth - log_r_estimated)**2, settings.trim_mean_fraction)), True
    
def median_error_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    errors = np.abs(log_r_truth - log_r_estimated)
    return np.median(errors)

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return np.sqrt(mean_squared_error(expected_log_r_truth, expected_log_r_estimated))
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return np.sqrt(mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites])), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(mean_squared_error(t_truth, t_estimated))

def trimmed_mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return np.sqrt(trim_mean(np.linalg.norm(t_truth - t_estimated,axis=1)**2, settings.trim_mean_fraction))
    
def median_error_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    errors = np.abs(t_truth - t_estimated)
    return np.median(errors)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')[settings.thetas_train]
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')[settings.thetas_train]
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.sqrt(np.var(expected_log_r_truth - expected_log_r_estimated))

# Main tables

In [38]:
labels = [r'AFC (density est.\ in $\boldx$)',
          'carl (PbP, raw)',
          'carl (PbP, cal.)',
          'carl (param., raw)',
          'carl (param., cal.)',
          'carl + score (param., raw)',
          'carl + score (param., cal.)',
          'SM score regression',
          'Ratio regression (PbP, raw)',
          'Ratio regression (PbP, cal.)',
          'Ratio regression (param., raw)',
          'Ratio regression (param., cal.)',
          r'Ratio + score regr.\ (param., raw)',
          r'Ratio + score regr.\ (param., cal.)']

folders = (['afc']
          + ['point_by_point'] * 2 + ['parameterized'] * 4
          + ['score_regression']
          + ['point_by_point'] * 2 + ['parameterized'] * 4)

filenames = ['afc',
             'carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'combined',
             'combined_calibrated',
             'scoreregression',
             'regression',
             'regression_calibrated',
             'regression',
             'regression_calibrated',
             'combinedregression',
             'combinedregression_calibrated']


def show_main_table(algorithm_begin=0, algorithm_end=None):
    
    #table = TablePrinter([trimmed_mse_log_r_nottrained, mse_expected_log_r],
    #                      precisions=[3,2])
    table = TablePrinter([expected_mse_log_r, expected_trimmed_mse_log_r],
                          precisions=[4,4])

    for i, (label, filename, folder) in enumerate(
        zip(labels[algorithm_begin:algorithm_end], filenames[algorithm_begin:algorithm_end], folders[algorithm_begin:algorithm_end])):

        if i > 0:
            table.new_block()

        if folder == 'point_by_point':
            table.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
            table.add('', 'PbP (2)', filename, 'point_by_point')
            table.add('', 'PbP (3)', filename + '_deep', 'point_by_point')

        elif folder == 'afc':
            table.add(label, '2d, $\epsilon = 0.4$ (2)', filename + '_2d_epsilon_0.40', 'afc')
            table.add('', '2d, $\epsilon = 0.2$ (2)', filename + '_2d_epsilon_0.20', 'afc')
            table.add('', '2d, $\epsilon = 0.1$ (2)', filename + '_2d_epsilon_0.10', 'afc')
            table.add('', '2d, $\epsilon = 0.05$ (2)', filename + '_2d_epsilon_0.05', 'afc')
            table.add('', '2d, $\epsilon = 0.02$ (2)', filename + '_2d_epsilon_0.02', 'afc')
            table.add('', '2d, $\epsilon = 0.01$ (2)', filename + '_2d_epsilon_0.01', 'afc')

            table.add('', '5d, $\epsilon = 0.4$ (2)', filename + '_5d_epsilon_0.40', 'afc')
            table.add('', '5d, $\epsilon = 0.2$ (2)', filename + '_5d_epsilon_0.20', 'afc')
            table.add('', '5d, $\epsilon = 0.1$ (2)', filename + '_5d_epsilon_0.10', 'afc')
            table.add('', '5d, $\epsilon = 0.05$ (2)', filename + '_5d_epsilon_0.05', 'afc')
            table.add('', '5d, $\epsilon = 0.02$ (2)', filename + '_5d_epsilon_0.02', 'afc')
            table.add('', '5d, $\epsilon = 0.01$ (2)', filename + '_5d_epsilon_0.01', 'afc')

        elif folder == 'score_regression':
            table.add(label, r'Fixed density est. on $t$  (2)', filename + '_score', 'score_regression')
            table.add('', r'Dyn. dens. est. on $t$  (2)', filename + '_rotatedscore', 'score_regression')
            table.add('', r'Dens. est. on $t \cdot \theta$  (2)', filename + '_scoretheta', 'score_regression')

        else:
            table.add(label, r'Baseline (1)', filename + '_shallow')
            table.add('', r'Baseline (2)', filename)
            table.add('', r'Baseline (3)', filename + '_deep')
            
            
            table.add('', r'Baseline, $\alpha = 0.01$ (2)', filename + '_alpha_0.010')
            table.add('', r'Baseline, $\alpha = 0.03$ (2)', filename + '_alpha_0.030')
            table.add('', r'Baseline, $\alpha = 0.3$ (2)', filename + '_alpha_0.30')
            table.add('', r'Baseline, $\alpha = 1$ (2)', filename + '_alpha_1.0')
            table.add('', r'Baseline, $\alpha = 3$ (2)', filename + '_alpha_3.0')
            table.add('', r'Baseline, $\alpha = 10$ (2)', filename + '_alpha_10')
            table.add('', r'Baseline, $\alpha = 30$ (2)', filename + '_alpha_30')
            table.add('', r'Baseline, $\alpha = 300$ (2)', filename + '_alpha_300')
            table.add('', r'Baseline, $\alpha = 1000$ (2)', filename + '_alpha_1000')
            
            table.add('', r'Baseline, large batches (2)', filename + '_largebatch')
            table.add('', r'Baseline, small batches (2)', filename + '_smallbatch')
            table.add('', r'Baseline, const.\ LR (2)', filename + '_constantlr')
            table.add('', r'Baseline, const.\ LR, large batches (2)', filename + '_constantlr_largebatch')
            table.add('', r'Baseline, const.\ LR, small batches (2)', filename + '_constantlr_largebatch')
            table.add('', r'Baseline, small LR (2)', filename + '_slowlearning')
            table.add('', r'Baseline, small LR, large batches (2)', filename + '_slowlearning_largebatch')
            table.add('', r'Baseline, small LR, small batches (2)', filename + '_slowlearning_smallbatch')
            table.add('', r'Baseline, small const.\ LR (2)', filename + '_slowlearning_constantlr')
            table.add('', r'Baseline, small const.\ LR, large batches (2)', filename + '_slowlearning_constantlr_largebatch')
            table.add('', r'Baseline, small const.\ LR, small batches (2)', filename + '_slowlearning_constantlr_smallbatch')
            table.add('', r'Baseline, large LR (2)', filename + '_fastlearning')
            table.add('', r'Baseline, large LR, large batches (2)', filename + '_fastlearning_largebatch')
            table.add('', r'Baseline, large LR, small batches (2)', filename + '_fastlearning_smallbatch')
            table.add('', r'Baseline, large const.\ LR (2)', filename + '_fastlearning_constantlr')
            table.add('', r'Baseline, large const.\ LR, large batches (2)', filename + '_fastlearning_constantlr_largebatch')
            table.add('', r'Baseline, large const.\ LR, small batches (2)', filename + '_fastlearning_constantlr_smallbatch')
            
            table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
            table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
            table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')
            
            table.add('', r'Aware, baseline (1)', filename + '_aware_shallow')
            table.add('', r'Aware, baseline (2)', filename + '_aware')
            table.add('', r'Aware, baseline (3)', filename + '_aware_deep')
            table.add('', r'Aware, baseline, SM dev. (2)', filename + '_aware_factorsm')
            table.add('', r'Aware, baseline, small LR (2)', filename + '_aware_slowlearning')
            table.add('', r'Aware, baseline, small LR, SM dev. (2)', filename + '_aware_factorsm_slowlearning')
            
            table.add('', r'Aware, basis (2)', filename + '_aware_basis')

    print(table.print())

In [39]:
show_main_table(0,5)

   \midrule
   \midrule
   \midrule
   carl (param., raw) & Baseline (1) & \emph{0.0254} & \emph{0.0060}\\
    & Baseline (2) & 0.0288 & 0.0069\\
   \midrule



In [40]:
show_main_table(5,8)

   \midrule
   \midrule
   SM score regression & Fixed density est. on $t$  (2) & \emph{0.0311} & \emph{0.0019}\\
    & Dyn. dens. est. on $t$  (2) & 0.0313 & 0.0020\\
    & Dens. est. on $t \cdot \theta$  (2) & 0.0585 & 0.0038\\



In [41]:
show_main_table(8,12)

   \midrule
   \midrule
   \midrule



In [22]:
show_main_table(12,None)

   \midrule



# Score accuracy table

In [17]:
labels = ['carl',
          'carl + score',
          'Ratio regression',
          'Ratio + score regr.']

filenames = ['carl',
             'combined',
             'regression',
             'combinedregression']

table = TablePrinter([trimmed_mse_score_nottrained],
                      precisions=[2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table.new_block()
        
    table.add(label, r'Baseline (1)', filename + '_shallow')
    table.add('', r'Baseline (2)', filename)
    table.add('', r'Baseline (3)', filename + '_deep')

    table.add('', r'Baseline, $\alpha = 0.01$ (2)', filename + '_alpha_0.010')
    table.add('', r'Baseline, $\alpha = 0.03$ (2)', filename + '_alpha_0.030')
    table.add('', r'Baseline, $\alpha = 0.3$ (2)', filename + '_alpha_0.30')
    table.add('', r'Baseline, $\alpha = 1$ (2)', filename + '_alpha_1.0')
    table.add('', r'Baseline, $\alpha = 3$ (2)', filename + '_alpha_3.0')
    table.add('', r'Baseline, $\alpha = 10$ (2)', filename + '_alpha_10')
    table.add('', r'Baseline, $\alpha = 30$ (2)', filename + '_alpha_30')
    table.add('', r'Baseline, $\alpha = 300$ (2)', filename + '_alpha_300')
    table.add('', r'Baseline, $\alpha = 1000$ (2)', filename + '_alpha_1000')

    table.add('', r'Baseline, large batches (2)', filename + '_largebatch')
    table.add('', r'Baseline, small batches (2)', filename + '_smallbatch')
    table.add('', r'Baseline, const.\ LR (2)', filename + '_constantlr')
    table.add('', r'Baseline, const.\ LR, large batches (2)', filename + '_constantlr_largebatch')
    table.add('', r'Baseline, const.\ LR, small batches (2)', filename + '_constantlr_largebatch')
    table.add('', r'Baseline, small LR (2)', filename + '_slowlearning')
    table.add('', r'Baseline, small LR, large batches (2)', filename + '_slowlearning_largebatch')
    table.add('', r'Baseline, small LR, small batches (2)', filename + '_slowlearning_smallbatch')
    table.add('', r'Baseline, small const.\ LR (2)', filename + '_slowlearning_constantlr')
    table.add('', r'Baseline, small const.\ LR, large batches (2)', filename + '_slowlearning_constantlr_largebatch')
    table.add('', r'Baseline, small const.\ LR, small batches (2)', filename + '_slowlearning_constantlr_smallbatch')
    table.add('', r'Baseline, large LR (2)', filename + '_fastlearning')
    table.add('', r'Baseline, large LR, large batches (2)', filename + '_fastlearning_largebatch')
    table.add('', r'Baseline, large LR, small batches (2)', filename + '_fastlearning_smallbatch')
    table.add('', r'Baseline, large const.\ LR (2)', filename + '_fastlearning_constantlr')
    table.add('', r'Baseline, large const.\ LR, large batches (2)', filename + '_fastlearning_constantlr_largebatch')
    table.add('', r'Baseline, large const.\ LR, small batches (2)', filename + '_fastlearning_constantlr_smallbatch')

    table.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    table.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    table.add('', r'Aware, baseline (1)', filename + '_aware_shallow')
    table.add('', r'Aware, baseline (2)', filename + '_aware')
    table.add('', r'Aware, baseline (3)', filename + '_aware_deep')
    table.add('', r'Aware, baseline, SM dev. (2)', filename + '_aware_factorsm')
    table.add('', r'Aware, baseline, small LR (2)', filename + '_aware_slowlearning')
    table.add('', r'Aware, baseline, small LR, SM dev. (2)', filename + '_aware_factorsm_slowlearning')

    table.add('', r'Aware, basis (2)', filename + '_aware_basis')

print(table.print())

   carl & Baseline (1) & 0.47\\
    & Baseline (2) & 0.39\\
    & Baseline (3) & 0.61\\
    & Baseline, large batches (2) & 0.30\\
    & Baseline, small batches (2) & 0.54\\
    & Baseline, const.\ LR (2) & 0.79\\
    & Baseline, const.\ LR, large batches (2) & 0.67\\
    & Baseline, const.\ LR, small batches (2) & 0.67\\
    & Baseline, small LR (2) & 0.21\\
    & Baseline, small LR, large batches (2) & 0.22\\
    & Baseline, small LR, small batches (2) & \emph{0.19}\\
    & Baseline, small const.\ LR (2) & 0.24\\
    & Baseline, small const.\ LR, large batches (2) & 0.21\\
    & Baseline, small const.\ LR, small batches (2) & 0.26\\
    & Baseline, large LR (2) & 2.94\\
    & Baseline, large LR, large batches (2) & 1.67\\
    & Baseline, large LR, small batches (2) & 2.22\\
    & Baseline, large const.\ LR (2) & 2.68\\
    & Baseline, large const.\ LR, large batches (2) & 2.50\\
    & Baseline, large const.\ LR, small batches (2) & 2.68\\
    & Random $\boldtheta$ (1) & 0.45\\
    & 